In [ ]:
!pip install gtts
!pip install playsound
!pip install tensorflow
!pip install opencv-python

In [ ]:
import os
import string
import glob
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
#import tensorflow.keras.applications.inception_v3
from tqdm import tqdm
import tensorflow.keras.preprocessing.image
import pickle
from time import time
import numpy as np
#from PIL import Image
import cv2
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import (LSTM, Embedding,TimeDistributed, Dense, RepeatVector,Activation, Flatten, Reshape, concatenate,Dropout, BatchNormalization)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import add
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from twilio.rest import Client
#import matplotlib.pyplot as plt
from gtts import gTTS
from playsound import playsound
START = "startseq"
STOP = "endseq"
EPOCHS = 10

In [ ]:
root_captioning = "M:\mini\data\data"
files_path = "M:\mini\jupyter\pickles"

In [ ]:
#1
null_punct = str.maketrans('', '', string.punctuation)
lookup = dict()

with open( os.path.join(root_captioning,'text','Flickr8k.token.txt'), 'r') as fp:
  
  max_length = 0
  for line in fp.read().split('\n'):
    tok = line.split()
    if len(line) >= 2:
      id = tok[0].split('.')[0]
      desc = tok[1:]
      desc = [word.lower() for word in desc]
      desc = [w.translate(null_punct) for w in desc]
      desc = [word for word in desc if len(word)>1]
      desc = [word for word in desc if word.isalpha()]
      max_length = max(max_length,len(desc))
      
      if id not in lookup:
        lookup[id] = list()
      lookup[id].append(' '.join(desc))
      
lex = set()
for key in lookup:
  [lex.update(d.split()) for d in lookup[key]]
print("done")
print(lookup)

In [ ]:
print(len(lookup)) 
print(len(lex)) 
print(max_length) 


In [ ]:
#2
img = glob.glob(os.path.join(root_captioning,'images', '*.jpg'))
print("done")
print(img[0],"    ",img[1])
print(len(img))

In [ ]:
len(img)

In [ ]:
train_images_path = os.path.join(root_captioning,'text','Flickr_8k.trainImages.txt') 
train_images = set(open(train_images_path, 'r').read().strip().split('\n'))
test_images_path = os.path.join(root_captioning,'text','Flickr_8k.testImages.txt') 
test_images = set(open(test_images_path, 'r').read().strip().split('\n'))

train_img = []
test_img = []

for i in img:
  f = os.path.split(i)[-1]
  if f in train_images: 
    train_img.append(f) 
  elif f in test_images:
    test_img.append(f) 

In [ ]:
print(len(train_images))
print(len(test_images))

In [ ]:
#3
train_descriptions = {k:v for k,v in lookup.items() if f'{k}.jpg' in train_images}
for n,v in train_descriptions.items(): 
  for d in range(len(v)):
    v[d] = f'{START} {v[d]} {STOP}'

In [ ]:
#print(train_descriptions)
len(train_descriptions)
#print(lookup)
for n,v in train_descriptions.items(): 
  print(lookup[n])
  break
test_descriptions = {k:v for k,v in lookup.items() if f'{k}.jpg' in test_images}
for n,v in test_descriptions.items():
  print(lookup[n])
  break


In [ ]:
encode_model = InceptionV3(weights='imagenet')
encode_model = Model(encode_model.input, encode_model.layers[-2].output)
WIDTH = 299
HEIGHT = 299
OUTPUT_DIM = 2048
preprocess_input = tensorflow.keras.applications.inception_v3.preprocess_input

In [ ]:
encode_model.summary()

In [ ]:
#5
def encodeImage(img):

  #img = img.resize(tuple(WIDTH, HEIGHT), Image.ANTIALIAS)
  img=cv2.resize(image,(WIDTH, HEIGHT),3, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
  x = tensorflow.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  x = encode_model.predict(x) 
  x = np.reshape(x, OUTPUT_DIM )
  return x

In [ ]:

train_path = os.path.join(files_path,f'train{OUTPUT_DIM}.pkl')
if not os.path.exists(train_path):
  start = time()
  encoding_train = {}
  for id in tqdm(train_img):
    image_path = os.path.join(root_captioning,'images', id)
    img = tensorflow.keras.preprocessing.image.load_img(image_path, \
            target_size=(HEIGHT, WIDTH))
    encoding_train[id] = encodeImage(img)
  with open(train_path, "wb") as fp:
    pickle.dump(encoding_train, fp)
else:
  with open(train_path, "rb") as fp:
    encoding_train = pickle.load(fp)

In [ ]:
test_path = os.path.join(files_path,f'test{OUTPUT_DIM}.pkl')
if not os.path.exists(test_path):
  start = time()
  encoding_test = {}
  for id in tqdm(test_img):
    image_path = os.path.join(root_captioning,'images', id)
    img = tensorflow.keras.preprocessing.image.load_img(image_path, \
                target_size=(HEIGHT, WIDTH))
    encoding_test[id] = encodeImage(img)
  with open(test_path, "wb") as fp:
    pickle.dump(encoding_test, fp)
else:
  with open(test_path, "rb") as fp:
    encoding_test = pickle.load(fp)

In [ ]:
print(encoding_test)

In [ ]:
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

In [ ]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d ==> %d' % (len(word_counts), len(vocab)))

In [ ]:
idxtoword = {}
wordtoidx = {}

ix = 1
for w in vocab:
    wordtoidx[w] = ix
    idxtoword[ix] = w
    ix += 1
    
vocab_size = len(idxtoword) + 1 
vocab_size


In [ ]:
print(idxtoword)

In [ ]:
idxtoword_path = os.path.join(files_path,'idxtoword.pkl')
with open(idxtoword_path, "wb") as fp:
    pickle.dump(idxtoword, fp)

In [ ]:
wordtoidx_path = os.path.join(files_path,'wordtoidx.pkl')
with open(wordtoidx_path, "wb") as fp:
    pickle.dump(wordtoidx, fp)

In [ ]:
max_length +=2
print(max_length)

In [ ]:
def data_generator(descriptions, photos, wordtoidx, \
                   max_length, num_photos_per_batch):
  x1, x2, y = [], [], []
  n=0
  while True:
    for key, desc_list in descriptions.items():
      n+=1
      photo = photos[key+'.jpg']
      for desc in desc_list:
        seq = [wordtoidx[word] for word in desc.split(' ') \
               if word in wordtoidx]
        for i in range(1, len(seq)):
          in_seq, out_seq = seq[:i], seq[i]
          in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
          out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
          x1.append(photo)
          x2.append(in_seq)
          y.append(out_seq)
      if n==num_photos_per_batch:
        yield ([np.array(x1), np.array(x2)], np.array(y))
        x1, x2, y = [], [], []
        n=0

In [ ]:
glove_dir = os.path.join(root_captioning)
embeddings_index = {} 
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print(f'Found {len(embeddings_index)} word vectors.')

In [ ]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoidx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
embedding_matrix_path = os.path.join(files_path,'embedding_matrix.pkl')
with open(embedding_matrix_path, "wb") as fp:
    pickle.dump(embedding_matrix, fp)

In [ ]:
inputs1 = Input(shape=(OUTPUT_DIM,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
caption_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
embedding_dim

In [ ]:
caption_model.summary()

In [ ]:
caption_model.layers[2].set_weights([embedding_matrix])
caption_model.layers[2].trainable = False
caption_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
model_path = os.path.join("M:\mini\jupyter\model",f'caption-model.hdf5')
if not os.path.exists(model_path):
  for i in tqdm(range(EPOCHS*2)):
      generator = data_generator(train_descriptions, encoding_train, 
                    wordtoidx, max_length, number_pics_per_bath)
      caption_model.fit_generator(generator, epochs=1,
                    steps_per_epoch=steps, verbose=1)

  caption_model.optimizer.lr = 1e-4
  number_pics_per_bath = 6
  steps = len(train_descriptions)//number_pics_per_bath

  for i in range(EPOCHS):
      generator = data_generator(train_descriptions, encoding_train, 
                    wordtoidx, max_length, number_pics_per_bath)
      caption_model.fit_generator(generator, epochs=1, 
                            steps_per_epoch=steps, verbose=1)  
  caption_model.save_weights(model_path)
else:
  caption_model.load_weights(model_path)
    

In [ ]:
def generateCaption(photo):
    in_text = START
    for i in range(max_length):
        sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = caption_model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = idxtoword[yhat]
        in_text += ' ' + word
        if word == STOP:
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
model=load_model(r'M:\mini\jupyter\model\alert_model.h5')

In [ ]:
import cv2
from datetime import datetime
vidcap = cv2.VideoCapture(r'C:\Users\yuvasri\Downloads\queda.mp4')
count = 0
success = True
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
T = datetime.now()
print(T)
text=[]
while success:
    success,image = vidcap.read()
    print("1")
    x1 = np.expand_dims(image, axis=0)
    x1 = preprocess_input(x1)
    outcome = [np.argmax(model.predict(x1))]
    if outcome==[[1]]:
        print("no")
        account_sid = "AC99e30cc80bd4de0e2946118bf1d1d1b9"
        auth_token = "5d737b2efe40561722780f53cd0fe1b0"
        client = Client(account_sid, auth_token)

        call = client.calls.create(
                        url='http://demo.twilio.com/docs/voice.xml',
                        to='+919182544970',
                        from_='+16503833718'
                    )
        print(call.sid)
    if count%(2*fps) == 0 :
         cv2.imwrite(r'M:\mini\jupyter\images\frame%d.jpg'%count,image)
         #print(image.shape)
         #print(type(image))
         #cv2.imshow("image"+str(count),image)
         #print('successfully written 10th frame')
         WIDTH = 299
         HEIGHT = 299
         a=(299,299)
         OUTPUT_DIM = 2048
         #img = np.resize(img,(299,299))
         img=cv2.resize(image,(299,299),3, fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
         img=encodeImage(img).reshape((1,OUTPUT_DIM))
         x=generateCaption(img)
         seconds=datetime.now()-T
         print(seconds.total_seconds())
         #x = image.img_to_array(image)
         if(seconds.total_seconds()>=10):
                T=datetime.now()
                print(x)
                text.append(x)
                print(" ".join(text))
                speech = gTTS(text="            ".join(text), lang='en', slow=False)
                speech.save(r'M:\mini\jupyter\audio\audio%d.mp3'%count)
                playsound(r'M:\mini\jupyter\audio\audio%d.mp3'%count)
                text=[]
         else:
                print(x)
                text.append(x)
        
    count+=1
    if cv2.waitKey(20) & 0xFF == ord('q'):
          break

cap.release()
cv2.destroyAllWindows()


In [ ]:
count=240
playsound(r'M:\mini\jupyter\audio\audio%d.mp3'%count)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
import tensorflow as tf
model=load_model('M:\mini\jupyter\model\alert_model.h5')
from twilio.rest import Client
img = image.load_img(r"C:\Users\jyoth\Desktop\6.jfif", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
outcome = [np.argmax(model.predict(x))]
if outcome==[[0]]:
    print("no")
    account_sid = "AC99e30cc80bd4de0e2946118bf1d1d1b9"
    auth_token = "5d737b2efe40561722780f53cd0fe1b0"
    client = Client(account_sid, auth_token)

    call = client.calls.create(
                        url='http://demo.twilio.com/docs/voice.xml',
                        to='+17704673802',
                        from_='+919182544970'
                    )

    print(call.sid)


In [ ]:
pip show twilio